import pandas as pd

from openpyxl import load_workbook

from openpyxl.styles import PatternFill

import random

import time

import logging

from geopy.distance import geodesic

\# Setup logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s -
%(levelname)s - %(message)s')

\# Define locations with geocoordinates for simulation

location_coords = {

'Freetown': (8.484444, -13.234444),

'Bo': (7.964722, -11.738333),

'Kenema': (7.876667, -11.1875),

'Makeni': (8.886111, -12.044167),

'Koidu': (8.644444, -10.971111)

}

\# Simulate ambulance availability

ambulances = {

'Freetown': 20,

'Bo': 20,

'Kenema': 20,

'Makeni': 20,

'Koidu': 17

}

\# Define scoring mappings with adjusted thresholds

vital_sign_ranges = {

'Heart Rate': \[(150, 5), (120, 3), (80, 1), (60, 0), (40, 3), (0, 5)\],

'Blood Pressure': \[(180, 5), (160, 3), (140, 2), (120, 1), (90, 0),
(70, 3), (0, 5)\],

'Respiratory Rate': \[(30, 5), (25, 3), (20, 2), (15, 1), (12, 0), (10,
3), (0, 5)\],

'Oxygen Saturation': \[(100, 0), (95, 1), (90, 3), (85, 5), (0, 5)\],

'Temperature': \[(40, 5), (39, 3), (38, 2), (37, 1), (36, 0), (35, 3),
(0, 5)\]

}

neurotrauma_scores = {

'GCS': \[(8, 4), (12, 2), (15, 0)\],

'Pupil Reactivity': {'both_non_reactive': 4, 'sluggish_or_unequal': 2,
'both_reactive': 0}

}

limb_mobility_scores = {'normal': 0, 'weakness_one_limb': 1,
'weakness_multiple_limbs': 2, 'paralysis': 4}

additional_signs_scores = {'Seizures': 2, 'Altered Consciousness': 3,
'Severe Headache or Vomiting': 1}

pain_response_scores = {'normal': 0, 'localized': 1, 'no_response': 4}

motor_strength_scores = {'normal': 0, 'moderate_weakness': 2,
'no_movement': 4}

spinal_cord_injury_scores = {'none': 0,
'pain_without_neurological_deficit': 2,
'pain_with_neurological_deficit': 4}

def calculate_score(value, mapping):

if isinstance(mapping, dict):

return mapping.get(value, 0)

else:

for threshold, points in mapping:

if value \>= threshold:

return points

return 0

def calculate_total_triage_score(patient_data):

scores = {

'vital_signs': sum(calculate_score(patient_data\[sign\], thresholds) for
sign, thresholds in vital_sign_ranges.items()),

'neurotrauma': calculate_score(patient_data\['GCS'\],
neurotrauma_scores\['GCS'\]) + calculate_score(patient_data\['Pupil
Reactivity'\], neurotrauma_scores\['Pupil Reactivity'\]),

'limb_mobility': calculate_score(patient_data\['Limb Mobility'\],
limb_mobility_scores),

'additional_signs': sum(calculate_score(sign, additional_signs_scores)
for sign, present in patient_data.items() if present and sign in
additional_signs_scores),

'pain_response': calculate_score(patient_data\['Pain Response'\],
pain_response_scores),

'motor_strength': calculate_score(patient_data\['Motor Strength'\],
motor_strength_scores),

'spinal_cord_injury': calculate_score(patient_data\['Spinal Cord
Injury'\], spinal_cord_injury_scores)

}

total_score = sum(scores.values())

if total_score \>= 30:

priority = 'Critical Priority'

elif total_score \>= 20:

priority = 'High Priority'

elif total_score \>= 10:

priority = 'Medium Priority'

else:

priority = 'Low Priority'

return total_score, priority

def triage_patients(df):

\# Apply the scoring algorithm to each row

results = df.apply(lambda row: calculate_total_triage_score(row),
axis=1)

df\['Total_Score'\] = results.apply(lambda x: x\[0\])

df\['Priority'\] = results.apply(lambda x: x\[1\])

df\['Status'\] = df\['Priority'\].apply(lambda x: 'En Route' if x in
\['Critical Priority', 'High Priority'\] else 'Waiting')

return df

def estimate_travel_time(location):

\# Estimate travel time based on distance and average speed

hospital_coords = location_coords\['Freetown'\]

location_coords\_ = location_coords\[location\]

distance = geodesic(location_coords\_, hospital_coords).kilometers

average_speed = 40 \# Average speed of ambulances in km/h

travel_time = distance / average_speed

return travel_time

def send_notification(patient_name, location, priority):

travel_time = estimate_travel_time(location)

message = f"ALERT: {priority} neurotrauma case - Patient: {patient_name}
at {location}. NEMS ambulance dispatched. Estimated travel time to
Connaught Hospital: {travel_time:.2f} hours."

logging.info(message)

def allocate_ambulance(location):

if ambulances\[location\] \> 0:

ambulances\[location\] -= 1

return True

else:

return False

def generate_random_patient_data():

gcs = random.choices(\[15, 13, 8, 9, 14\], \[0.6, 0.2, 0.05, 0.1,
0.05\])\[0\]

pupil_reactivity = random.choices(\['both_reactive',
'sluggish_or_unequal', 'both_non_reactive'\], \[0.7, 0.2, 0.1\])\[0\]

limb_mobility = random.choices(\['normal', 'weakness_one_limb',
'paralysis', 'weakness_multiple_limbs'\], \[0.6, 0.25, 0.05, 0.1\])\[0\]

return {

'Name': f"Patient {random.randint(1, 1000)}",

'GCS': gcs,

'Pupil Reactivity': pupil_reactivity,

'Limb Mobility': limb_mobility,

'Heart Rate': random.randint(60, 100),

'Blood Pressure': random.randint(90, 140),

'Respiratory Rate': random.randint(12, 20),

'Oxygen Saturation': random.randint(90, 100),

'Temperature': round(random.uniform(36.0, 38.0), 1),

'Seizures': random.choice(\[True, False\]),

'Altered Consciousness': random.choice(\[True, False\]),

'Severe Headache or Vomiting': random.choice(\[True, False\]),

'Pain Response': random.choice(\['normal', 'localized',
'no_response'\]),

'Motor Strength': random.choice(\['normal', 'moderate_weakness',
'no_movement'\]),

'Spinal Cord Injury': random.choice(\['none',
'pain_without_neurological_deficit',
'pain_with_neurological_deficit'\]),

'Location': random.choice(list(location_coords.keys()))

}

def print_welcome_message():

print("Welcome to the Sierra Leone Neurotrauma Triage System (SLANT).")

print("This system helps in triaging neurotrauma cases based on various
medical parameters.")

print("\nInstructions for variables (English):")

print("1. GCS (Glasgow Coma Scale): Score to assess level of
consciousness.")

print("2. Pupil Reactivity: Reaction of pupils ('both_reactive',
'sluggish_or_unequal', 'both_non_reactive').")

print("3. Limb Mobility: Movement ability ('normal',
'weakness_one_limb', 'paralysis', 'weakness_multiple_limbs').")

print("4. Heart Rate: Beats per minute (e.g., 70).")

print("5. Blood Pressure: mmHg (e.g., 120).")

print("6. Respiratory Rate: Breaths per minute (e.g., 16).")

print("7. Oxygen Saturation: Percentage (e.g., 98).")

print("8. Temperature: Degrees Celsius (e.g., 37).")

print("9. Seizures: Presence of seizures (True/False).")

print("10. Altered Consciousness: Altered state of consciousness
(True/False).")

print("11. Severe Headache or Vomiting: Presence of severe headache or
vomiting (True/False).")

print("12. Pain Response: Response to pain ('normal', 'localized',
'no_response').")

print("13. Motor Strength: Muscle strength ('normal',
'moderate_weakness', 'no_movement').")

print("14. Spinal Cord Injury: Type of injury ('none',
'pain_without_neurological_deficit',
'pain_with_neurological_deficit').")

print("\nInstructions for variables (Krio):")

print("1. GCS (Glasgow Coma Scale): Score for check lek ow di posin dem
wake up.")

print("2. Pupil Reactivity: Di way eye den di act ('both_reactive',
'sluggish_or_unequal', 'both_non_reactive').")

print("3. Limb Mobility: Di way body part den di move ('normal',
'weakness_one_limb', 'paralysis', 'weakness_multiple_limbs').")

print("4. Heart Rate: Beats per minute (e.g., 70).")

print("5. Blood Pressure: mmHg (e.g., 120).")

print("6. Respiratory Rate: Di number of time dem di posin di breathe
per minute (e.g., 16).")

print("7. Oxygen Saturation: Percentage (e.g., 98).")

print("8. Temperature: Degree Celsius (e.g., 37).")

print("9. Seizures: If posin de shake (True/False).")

print("10. Altered Consciousness: If posin get correct sense
(True/False).")

print("11. Severe Headache or Vomiting: If posin get serious headache or
vomit (True/False).")

print("12. Pain Response: How posin di respond to pain ('normal',
'localized', 'no_response').")

print("13. Motor Strength: Di power of muscle ('normal',
'moderate_weakness', 'no_movement').")

print("14. Spinal Cord Injury: Di type of injury ('none',
'pain_without_neurological_deficit',
'pain_with_neurological_deficit').")

print("\nLet's begin the simulation...\n")

def print_dashboard(df):

print("\nCurrent Dashboard:")

print("-------------------------------------------------")

print(df\[\['Name', 'Location', 'Priority', 'Status'\]\])

print("-------------------------------------------------")

print("\nAmbulance Availability:")

for location, count in ambulances.items():

print(f"{location}: {count} available ambulances")

print("-------------------------------------------------\n")

def search_patient(df, patient_name):

patient_info = df\[df\['Name'\] == patient_name\]

if not patient_info.empty:

print("\nPatient Details:")

print("-------------------------------------------------")

with pd.option_context('display.max_rows', None, 'display.max_columns',
None):

print(patient_info.to_string(index=False))

print("-------------------------------------------------")

else:

print(f"No patient found with name {patient_name}.")

def main():

print_welcome_message()

\# Create an empty DataFrame

df = pd.DataFrame()

\# Simulation loop

try:

while True:

command = input("Enter command (simulate/search/exit):
").strip().lower()

if command == "simulate":

new_patient_data = generate_random_patient_data()

if allocate_ambulance(new_patient_data\['Location'\]):

df = pd.concat(\[df, pd.DataFrame(\[new_patient_data\])\],
ignore_index=True)

df = triage_patients(df)

\# Save the updated dataframe to an Excel file with conditional
formatting

output_file_path =
'/Users/taramenon/Downloads/updated_patient_data_colored.xlsx'

df.to_excel(output_file_path, index=False)

\# Apply conditional formatting to Excel file

wb = load_workbook(output_file_path)

ws = wb.active

\# Define colors for each priority level

priority_colors = {

'Critical Priority': 'FF0000', \# Red

'High Priority': 'FFA500', \# Orange

'Medium Priority': 'FFFF00', \# Yellow

'Low Priority': '00FF00' \# Green

}

\# Get the column letter for 'Priority'

priority_column = None

for col in ws.iter_cols(1, ws.max_column):

if col\[0\].value == 'Priority':

priority_column = col\[0\].column_letter

break

if priority_column:

for row in ws.iter_rows(min_row=2, max_row=ws.max_row):

cell = ws\[f'{priority_column}{row\[0\].row}'\]

priority = cell.value

if priority in priority_colors:

color = priority_colors\[priority\]

fill = PatternFill(start_color=color, end_color=color,
fill_type="solid")

for c in row:

c.fill = fill

wb.save(output_file_path)

\# Check and alert for new priority cases

patient_name = new_patient_data\['Name'\]

location = new_patient_data\['Location'\]

total_score, priority = calculate_total_triage_score(new_patient_data)

send_notification(patient_name, location, priority)

logging.info(f"Updated patient data saved to {output_file_path}")

\# Print the dashboard

print_dashboard(df)

else:

logging.warning(f"No available ambulances in
{new_patient_data\['Location'\]}. Patient {new_patient_data\['Name'\]}
is waiting.")

\# Wait for a few seconds before the next iteration to simulate
real-time data arrival

time.sleep(5)

elif command == "search":

patient_name = input("Enter patient name to search: ").strip()

search_patient(df, patient_name)

elif command == "exit":

break

else:

print("Invalid command. Please enter 'simulate', 'search', or 'exit'.")

except KeyboardInterrupt:

logging.info("Simulation stopped.")

if \_\_name\_\_ == "\_\_main\_\_":

main()